## Парсинг данных об автомобилях с пробегом (Москва) с сайта "auto.ru"
Файл parsing_auto_ru.ipynb. Предназначен для парсинга данных об автомобилях с сайта auto.ru
и создания тренировочного датасета train.csv.

In [1]:
import pandas as pd
import re
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool
import json  
from time import gmtime, strftime

### Марки автомобилей для парсинга

In [2]:
marks = ['bmw', 'mercedes', 'volkswagen', 'kia', 'hyundai', 'vaz',
        'nissan', 'ford', 'toyota', 'audi', 'skoda', 'opel',
        'chevrolet', 'mitsubishi', 'mazda', 'renault', 'land_rover', 'volvo',
        'peugeot', 'honda', 'infiniti', 'lexus', 'citroen', 'subaru',
        'porsche', 'ssang_yong', 'suzuki', 'gaz', 'daewoo', 'jeep',
        'mini', 'saab', 'acura', 'cadillac', 'jaguar', 'haval',
        'lifan', 'geely', 'ravon', 'seat', 'alfa_romeo', 'isuzu']

len(marks)

42

### Функции

In [3]:
def create_train_set(marks):
    """Главная функция. Принимает список марок машин.
    Возвращает список марок машин, полученных при парсинге
    и затраченное время в секундах"""
    begin = datetime.now()
    list_marks = parse_cars_parallel(marks)
    duration = (datetime.now() - begin).total_seconds()
    return [list_marks, duration]


def parse_cars_parallel(marks):
    """Многопоточный парсинг"""
    pool = Pool(6)
    result = pool.map(get_list_cars, marks)
    pool.close()
    pool.join()
    return result


def get_list_cars(mark):
    """Собирает все машины марки mark"""
    list_cars = []
    url_base = 'https://auto.ru/moskva/cars/'+ mark +'/used/?sort=price-asc'
    for n in range(6):
        if n == 0:
            url = url_base + '&km_age_to=50000&page='
        elif n == 1:
            url = url_base + '&km_age_from=50001&km_age_to=100000&page='
        elif n == 2:
            url = url_base + '&km_age_from=100001&km_age_to=150000&page='
        elif n == 3:
            url = url_base + '&km_age_from=150001&km_age_to=200000&page='
        elif n == 4:
            url = url_base + '&km_age_from=200001&km_age_to=250000&page='
        else:
            url = url_base + '&km_age_from=250001&page='
        for i in range(1, 100):
            url_page = url + str(i)
            cars = get_all_cars_on_page(url_page) # список словарей
            if len(cars) > 0:
                list_cars.extend(cars)
            else:
                break
    
    create_csv(list_cars, mark)
    return mark

    
def create_csv(list_cars, mark):
    """Создает csv-файл, содержащий машины марки mark"""
    cols = ['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
            'numberOfDoors', 'productionDate', 'vehicleConfiguration',
            'vehicleTransmission', 'engineDisplacement', 'enginePower', 'mileage',
            'Привод', 'Руль', 'Владельцы', 'ПТС', 'Комплектация', 'description',
            'Владение', 'seller_type', 'url', 'price']
    df = pd.DataFrame(list_cars , columns=cols)
    df.to_csv(f'{mark}.csv', index=False)
    t = strftime("%H:%M:%S", gmtime())
    print (t + ' ' + mark + '.csv создан')
    return 1


def get_all_cars_on_page(url):
    """Возвращает все машины на странице в виде списка словарей. Один словарь - одна машина"""
    # Пример url: 'https://auto.ru/moskva/cars/chevrolet/used/?sort=price-asc&km_age_from=80001&page=74'
    list_cars = []
    try:
        soup = get_soup(url)
        cars = soup.find('div', class_ = 'ListingCars-module__container ListingCars-module__list') \
        .find_all('div', class_ = 'ListingItem-module__container')
        for car in cars:
            car_info = get_car_info(car)
            if len(car_info) > 0:
                list_cars.append(car_info)      
    except:
        return [] # если пусто, то вернуть пустой список
    return list_cars


def get_car_info(car):
    """Принимает объект BeautifulSoup. Возвращает информацию о машине в виде словаря"""
    car_info = {}
    region = car.find('span', class_="MetroListPlace__regionName MetroListPlace_nbsp").text
    if region == 'Москва':
        for meta in car.find_all('meta'):
            # формируем словарь из свойств itemprop: {bodyType: 'седан', brand: 'CHEVROLET', ...}
            car_info[meta['itemprop']] = meta['content'].replace("\xa0", " ")
        more_data = get_info_on_page(car_info['url'])
        car_info = {**car_info, **more_data}
    return car_info


def get_info_on_page(url):
    """Собирает данные о машине и возвращает словарь"""
    try:    
        soup = get_soup(url)
        info = (soup.find(class_ = 'CardInfo')).find_all('li')
        list_li = [li.find_all('span') for li in info]
        car_info = {li[0].text.replace("\xa0", " "): li[1].text.replace("\xa0", " ") for li in list_li}
    except Exception:
        return {}

    try:
        info = soup.find(id="initial-state").text
        info = json.loads(info)
    except Exception:
        return car_info   
    try:
        car_info['mileage'] = info['card']['state']['mileage']
    except Exception:
        pass
    try:
        car_info['description'] = info['card']['description']
    except Exception:
        pass
    try:
        car_info['Комплектация'] = info['card']['vehicle_info']['equipmentGroups']
    except Exception:
        pass
    try:
        car_info['seller_type'] = info['card']['seller_type']
    except Exception:
        pass
    
    return car_info
    

def get_soup(url):
    """Возвращает объект BeautifulSoup по входной ссылке url"""
    r = requests.get(url)
    r.encoding = 'utf-8'
    return BeautifulSoup(r.text, 'html.parser')

### Запуск парсинга

In [4]:
res = create_train_set(marks)

03:46:53 skoda.csv создан
04:17:27 toyota.csv создан
04:20:33 nissan.csv создан
04:20:42 opel.csv создан
04:21:25 hyundai.csv создан
04:25:29 volkswagen.csv создан
04:42:00 bmw.csv создан
04:52:50 chevrolet.csv создан
05:04:27 audi.csv создан
05:14:06 ford.csv создан
05:19:26 vaz.csv создан
05:25:10 mitsubishi.csv создан
05:30:48 mazda.csv создан
05:35:03 kia.csv создан
05:36:26 peugeot.csv создан
05:39:13 land_rover.csv создан
05:40:23 infiniti.csv создан
05:46:02 citroen.csv создан
05:51:31 honda.csv создан
05:55:08 lexus.csv создан
05:56:02 renault.csv создан
05:57:01 subaru.csv создан
06:00:30 volvo.csv создан
06:01:45 mini.csv создан
06:01:52 acura.csv создан
06:02:06 porsche.csv создан
06:02:52 suzuki.csv создан
06:03:16 saab.csv создан
06:03:23 daewoo.csv создан
06:06:18 cadillac.csv создан
06:06:52 jaguar.csv создан
06:07:26 haval.csv создан
06:08:16 mercedes.csv создан
06:08:53 jeep.csv создан
06:09:16 gaz.csv создан
06:09:29 ssang_yong.csv создан


In [5]:
res[1] # Затраченное время в секундах

10065.765966

### Сбор всех полученных файлов в один 'train.csv'

In [14]:
marks = ['bmw', 'mercedes', 'volkswagen', 'kia', 'hyundai', 'vaz',
        'nissan', 'ford', 'toyota', 'audi', 'skoda', 'opel',
        'chevrolet', 'mitsubishi', 'mazda', 'renault', 'land_rover', 'volvo',
        'peugeot', 'honda', 'infiniti', 'lexus', 'citroen', 'subaru',
        'porsche', 'ssang_yong', 'suzuki', 'gaz', 'daewoo', 'jeep',
        'mini', 'saab', 'acura', 'cadillac', 'jaguar', 'haval',
        'lifan', 'geely', 'ravon', 'seat', 'alfa_romeo', 'isuzu']

train = pd.concat([pd.read_csv(marks[i] + ".csv") for i in range(len(marks))])

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68083 entries, 0 to 17
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              68083 non-null  object 
 1   brand                 68083 non-null  object 
 2   color                 68083 non-null  object 
 3   fuelType              68083 non-null  object 
 4   modelDate             68083 non-null  int64  
 5   name                  68083 non-null  object 
 6   numberOfDoors         68083 non-null  int64  
 7   productionDate        68083 non-null  int64  
 8   vehicleConfiguration  68083 non-null  object 
 9   vehicleTransmission   68083 non-null  object 
 10  engineDisplacement    68083 non-null  object 
 11  enginePower           68083 non-null  object 
 12  mileage               68061 non-null  float64
 13  Привод                68061 non-null  object 
 14  Руль                  68061 non-null  object 
 15  Владельцы             

In [16]:
train['url'].nunique()

68066

In [17]:
train2 = train.drop_duplicates()

In [20]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68066 entries, 0 to 17
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              68066 non-null  object 
 1   brand                 68066 non-null  object 
 2   color                 68066 non-null  object 
 3   fuelType              68066 non-null  object 
 4   modelDate             68066 non-null  int64  
 5   name                  68066 non-null  object 
 6   numberOfDoors         68066 non-null  int64  
 7   productionDate        68066 non-null  int64  
 8   vehicleConfiguration  68066 non-null  object 
 9   vehicleTransmission   68066 non-null  object 
 10  engineDisplacement    68066 non-null  object 
 11  enginePower           68066 non-null  object 
 12  mileage               68044 non-null  float64
 13  Привод                68044 non-null  object 
 14  Руль                  68044 non-null  object 
 15  Владельцы             

In [21]:
train2.to_csv('train.csv', index=False)

In [22]:
train2

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,Привод,Руль,Владельцы,ПТС,Комплектация,description,Владение,seller_type,url,price
0,седан,BMW,чёрный,бензин,1987,2.0 MT,4,1992,SEDAN MECHANICAL 2.0,механическая,...,задний,Левый,3 или более,Оригинал,NaN,"на ходу , требуется косметика и всё за год все...",NaN,PRIVATE,https://auto.ru/cars/used/sale/bmw/5er/1094556...,100000
1,седан,BMW,пурпурный,дизель,1987,2.5 MT,4,1992,SEDAN MECHANICAL 2.5,механическая,...,задний,Левый,3 или более,Оригинал,"[{'name': 'Обзор', 'values': ['Противотуманные...",Все вопросы по телефону!,NaN,PRIVATE,https://auto.ru/cars/used/sale/bmw/5er/1095955...,300000
2,хэтчбек 5 дв.,BMW,чёрный,бензин,2011,1.6 AT,5,2013,HATCHBACK_5_DOORS AUTOMATIC 1.6,автоматическая,...,задний,Левый,2 владельца,Оригинал,NaN,Продаю BMW 1. Самое надежное сочетание мотор -...,NaN,PRIVATE,https://auto.ru/cars/used/sale/bmw/1er/1096723...,650000
3,седан,BMW,чёрный,бензин,1949,2.0 MT,4,1950,SEDAN MECHANICAL 2.0,механическая,...,задний,Левый,3 или более,Оригинал,NaN,Предлагаю к продаже BMW 340. 90% - деталей ор...,11 лет и 2 месяца,PRIVATE,https://auto.ru/cars/used/sale/bmw/340/1090070...,650000
4,внедорожник 5 дв.,BMW,коричневый,бензин,2012,2.0 AT,5,2013,ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,...,задний,Левый,2 владельца,Оригинал,NaN,"Состояние идеальное. Крашенных деталей нет, бе...",NaN,PRIVATE,https://auto.ru/cars/used/sale/bmw/x1/10957139...,679000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,внедорожник 5 дв.,ISUZU,синий,бензин,1981,2.8 MT,5,1989,ALLROAD_5_DOORS MECHANICAL 2.8,механическая,...,полный,Левый,3 или более,Оригинал,NaN,"Рама хорошая, пороги требуют внимания.\nКраска...",NaN,PRIVATE,https://auto.ru/cars/used/sale/isuzu/trooper/1...,140000
14,внедорожник 5 дв.,ISUZU,серый,бензин,1981,2.6 MT,5,1988,ALLROAD_5_DOORS MECHANICAL 2.6,механическая,...,полный,Левый,3 или более,Оригинал,NaN,Полностью переварена и окрашена.\nВсе остальны...,1 год и 10 месяцев,PRIVATE,https://auto.ru/cars/used/sale/isuzu/trooper/1...,170000
15,внедорожник 5 дв.,ISUZU,белый,бензин,1998,3.2 AT,5,1999,ALLROAD_5_DOORS AUTOMATIC 3.2,автоматическая,...,полный,Левый,3 или более,Дубликат,"[{'name': 'Прочее', 'values': ['Защита картера...",Состояние кузова более чем достойное для своих...,3 года и 6 месяцев,PRIVATE,https://auto.ru/cars/used/sale/isuzu/rodeo/109...,255000
16,внедорожник 5 дв.,ISUZU,зелёный,бензин,1992,3.5 MT,5,2001,ALLROAD_5_DOORS MECHANICAL 3.5,механическая,...,полный,Левый,3 или более,Оригинал,"[{'name': 'Прочее', 'values': ['Защита картера...",Продаётся рaмный внедорожник ISUZU TROOPER\nБу...,6 лет и 1 месяц,PRIVATE,https://auto.ru/cars/used/sale/isuzu/trooper/1...,400000
